In [ ]:
# -*- coding: utf-8 -*-
import io, re, math, warnings, os
from datetime import datetime
import pandas as pd
import requests
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import streamlit as st

warnings.simplefilter("ignore", FutureWarning)

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperar variables inyectadas por app.py
palette = locals().get("active_palette", ["#0e1c2c", "#889064", "#ff9f18"])
active_font = locals().get("active_font", "sans-serif")

# Colores dinámicos adaptados de la paleta
COL_DISPONIBLE = palette[0]
COL_OCUPADO = palette[1] if len(palette) > 1 else palette[0]
COL_LINEA_OCUPACION = palette[2] if len(palette) > 2 else (palette[1] if len(palette) > 1 else palette[0])

# Configuración general
HTTP_HEADERS = {"User-Agent": "StreamlitApp/DataTurClient"}
URL_70 = "https://repodatos.atdt.gob.mx/s_turismo/ocupacion_hotelera/Base70centros.csv"
URL_HIST = None # URL para histórico 2000-2014, si existe

# =============================================================================
# 2. LECTURA Y UTILIDADES (CACHED)
# =============================================================================

# Constante de Fallback (CORREGIDA: el último elemento ahora usa 'occ' en lugar de 'ocup')
FALLBACK_CENTROS = {
    ('Centros de Playa','jul-24'): {'disp': 209_591, 'ocup': 144_520, 'occ': 69.0},
    ('Centros de Playa','jul-25'): {'disp': 207_478, 'ocup': 139_690, 'occ': 67.3},
    ('Ciudades','jul-24'): {'disp': 239_776, 'ocup': 126_853, 'occ': 52.9},
    ('Ciudades','jul-25'): {'disp': 240_941, 'ocup': 124_389, 'occ': 51.6},
}

def _get(url, **kw):
    """Realiza una solicitud HTTP con headers y timeout."""
    kw = {**dict(timeout=60, headers=HTTP_HEADERS), **kw}
    r = requests.get(url, **kw)
    r.raise_for_status()
    return r

def _sniff_text_and_sep(url):
    """Detecta codificación y separador a partir del contenido."""
    encodings = ["utf-16","utf-8-sig","utf-8","latin-1","cp1252"]
    raw = _get(url).content
    text = None
    for enc in encodings:
        try:
            text = raw.decode(enc)
            break
        except Exception: continue
    if text is None: text = raw.decode("utf-8", errors="ignore")
    first_line = next((ln for ln in text.splitlines() if ln.strip()), "")
    counts = {sep: first_line.count(sep) for sep in [';', ',', '\t', '\n']}
    sep = max(counts, key=counts.get) if counts else ','
    return text, sep

@st.cache_data(show_spinner="Descargando y limpiando CSV de DataTur...")
def read_csv_super(url):
    """Lector robusto de CSV."""
    text, sep = _sniff_text_and_sep(url)
    buf = io.StringIO(text)
    try:
        df = pd.read_csv(buf, sep=sep, engine='python', on_bad_lines='skip', low_memory=False, dtype=str)
        if df.shape[1] > 1: return df
    except Exception: pass

    # Intento de parseo manual
    rows = [r for r in text.splitlines() if r.strip()]
    if not rows: return pd.DataFrame()
    header_cells = [h.strip() for h in rows[0].split(sep)]
    data = [[c.strip() for c in r.split(sep)] for r in rows[1:]]
    df = pd.DataFrame(data, columns=header_cells)
    return df

# Lógica de columna mantenida
def make_clean_columns(cols):
    clean, seen = [], {}
    for c in cols:
        base = re.sub(r"\s+", " ", str(c).strip()).lower() or "col"
        if base not in seen: seen[base] = 0; clean.append(base)
        else: seen[base] += 1; clean.append(f"{base}__dup{seen[base]}")
    return clean

def collapse_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty: return df
    df = df.copy(); df.columns = make_clean_columns(df.columns)
    groups = {}
    for c in df.columns: base = c.split("__dup")[0]; groups.setdefault(base, []).append(c)
    out = pd.DataFrame(index=df.index)
    for base, cols in groups.items():
        if len(cols) == 1: out[base] = df[cols[0]]
        else: out[base] = df[cols].apply(lambda row: next((x for x in row if pd.notna(x) and str(x).strip() != ""), None), axis=1)
    return out

def series_1d(df: pd.DataFrame, col: str) -> pd.Series:
    obj = df[col]
    if isinstance(obj, pd.DataFrame):
        return obj.apply(lambda row: next((x for x in row if pd.notna(x) and str(x).strip() != ""), None), axis=1)
    return obj

def safe_to_numeric(df: pd.DataFrame, col: str) -> pd.Series:
    s = series_1d(df, col)
    return pd.to_numeric(s, errors='coerce')

@st.cache_data(show_spinner=False)
def std_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Normalización de nombres de columnas y conversión a tipos numéricos."""
    if df is None or df.empty: return df
    df = collapse_duplicate_columns(df)
    ren_map = {};
    for c in df.columns:
        lc = c.lower()
        if lc in ['año','ano'] or lc.startswith('año'): ren_map[c] = 'anio'
        elif lc == 'periodo': ren_map[c] = 'periodo'
        elif ('cuartos' in lc and 'dispon' in lc) or 'disponibles pr' in lc or 'disp_prom' in lc: ren_map[c] = 'cuartos_disponibles_pd'
        elif ('cuartos' in lc and 'ocup' in lc) or 'ocupados pr' in lc or 'ocup_prom' in lc: ren_map[c] = 'cuartos_ocupados_pd'
        elif ('porc' in lc and 'ocup' in lc) or 'porcentaje de ocupación' in lc or lc in ['% ocupacion','% ocupación']: ren_map[c] = 'porc_ocupacion'
        elif ('categoria' in lc) or ('categoría' in lc) or ('estrella' in lc) or ('clasificacion' in lc): ren_map[c] = 'categoria'
        elif ('tipo centro' in lc) or ('tipo_centro' in lc): ren_map[c] = 'tipo_centro'
        elif ('centro turístico' in lc) or ('centro_turistico' in lc) or lc == 'centro' or ('destino' in lc): ren_map[c] = 'centro_turistico'
        elif lc == 'mes' or re.search(r'\bmes\b', lc): ren_map[c] = 'mes'
        elif ('entidad' in lc) or ('estado' in lc): ren_map[c] = 'entidad'
    df = df.rename(columns=ren_map)
    df = collapse_duplicate_columns(df)

    meses_map = {'ene':1,'enero':1,'feb':2,'febrero':2,'mar':3,'marzo':3,'abr':4,'abril':4,'may':5,'mayo':5,'jun':6,'junio':6,'jul':7,'julio':7,'ago':8,'agosto':8,'sep':9,'sept':9,'septiembre':9,'oct':10,'octubre':10,'nov':11,'noviembre':11,'dic':12,'diciembre':12}
    if 'mes' in df.columns:
        df['mes'] = series_1d(df, 'mes').apply(lambda x: meses_map.get(str(x).strip().lower(), x))

    for col in ['anio','mes','cuartos_disponibles_pd','cuartos_ocupados_pd','porc_ocupacion']:
        if col in df.columns: df[col] = safe_to_numeric(df, col)

    if 'anio' in df.columns: df = df[~df['anio'].isna()]
    return df

@st.cache_data(show_spinner=False)
def annual_national(df: pd.DataFrame) -> pd.DataFrame:
    """Agregación de cuartos disponibles/ocupados a nivel nacional anual."""
    req = {'anio','cuartos_disponibles_pd','cuartos_ocupados_pd'}
    if df.empty or not req.issubset(set(df.columns)): return pd.DataFrame(columns=['anio','cuartos_disponibles_pd','cuartos_ocupados_pd','porc_ocupacion'])
    agg = df.groupby(['anio']).agg(
        cuartos_disponibles_pd=('cuartos_disponibles_pd','sum'),
        cuartos_ocupados_pd=('cuartos_ocupados_pd','sum')
    ).reset_index()
    agg['porc_ocupacion'] = (agg['cuartos_ocupados_pd'] / agg['cuartos_disponibles_pd'] * 100).replace([math.inf, -math.inf], pd.NA)
    return agg

@st.cache_data(show_spinner=False)
def category_occupancy_latest_year(df: pd.DataFrame):
    """Agregación de ocupación por categoría para el último año con datos."""
    if df.empty or 'anio' not in df.columns: return None, pd.DataFrame()
    y = int(df['anio'].dropna().max())
    if 'categoria' not in df.columns: return y, pd.DataFrame()
    sub = df[df['anio']==y].copy()
    req = {'cuartos_disponibles_pd','cuartos_ocupados_pd'}
    if not req.issubset(set(sub.columns)): return y, pd.DataFrame()
    cat = sub.groupby('categoria').agg(
        cuartos_disponibles_pd=('cuartos_disponibles_pd','sum'),
        cuartos_ocupados_pd=('cuartos_ocupados_pd','sum')
    ).reset_index()
    if cat.empty: return y, cat
    cat['porc_ocupacion'] = cat['cuartos_ocupados_pd'] / cat['cuartos_disponibles_pd'] * 100

    def cat_key(x):
        try: n = re.findall(r"\d+", str(x)); return int(n[0]) if n else 0
        except: return 0
    cat['__sort'] = cat['categoria'].apply(cat_key)
    cat = cat.sort_values(['__sort','categoria']).drop(columns='__sort')
    return y, cat

def select_years_for_plot(annual_df: pd.DataFrame, prefer='quinquenios', min_points=3, last_n=10):
    """Selecciona años para plotear (mantenida la lógica original)."""
    if annual_df.empty or 'anio' not in annual_df.columns: return []
    years = sorted(annual_df['anio'].dropna().unique().astype(int))
    if not years: return []
    if prefer == 'quinquenios':
        start = 2000 if min(years) <= 2000 else min(years) - (min(years) % 5)
        quin = [y for y in range(start, max(years)+1, 5) if y in years]
        if len(quin) >= min_points: return quin
    last = [y for y in years if y >= max(years) - (last_n - 1)]
    if len(last) >= min_points: return last
    return years

# -----------------------------------------------------------------------------
# 3. PLOTEO STREAMLIT (CENTRADO y Colores)
# -----------------------------------------------------------------------------

def _base_layout(title_text):
    """Layout base con título centrado y estilo de fuente."""
    return dict(
        title=dict(
            text=title_text,
            x=0.5,            # <--- TÍTULO CENTRADO
            xanchor='center',
            font=dict(size=18, family=active_font)
        ),
        font=dict(family=active_font, size=12, color='#000'),
        paper_bgcolor='white',
        plot_bgcolor='white',
        margin=dict(l=60, r=40, t=70, b=120),
        legend=dict(orientation='h', yanchor='top', y=-0.18, xanchor='center', x=0.5, bgcolor='rgba(255,255,255,0)'),
    )

def plot_hist_barras_con_linea(annual_df: pd.DataFrame, prefer='quinquenios'):
    """Gráfica 1: Histórico anual (Barras Cuartos + Línea % Ocupación) CON TABLA FIJA."""
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if annual_df.empty or 'anio' not in annual_df.columns:
        st.warning("No hay datos disponibles para la serie histórica nacional.")
        return

    years = select_years_for_plot(annual_df, prefer=prefer, min_points=3, last_n=10)
    dfp = annual_df[annual_df['anio'].isin(years)].sort_values('anio').copy()

    if dfp.empty:
        st.warning("No hay años seleccionados para graficar el histórico.")
        return

    x = [str(int(y)) for y in dfp['anio']]

    # Barras (miles)
    fig.add_trace(
        go.Bar(
            x=x, y=dfp['cuartos_disponibles_pd']/1000,
            name='Cuartos disponibles (miles)',
            marker_color=COL_DISPONIBLE,
            hovertemplate="Disponibles: %{y:,.0f} mil<extra></extra>"
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Bar(
            x=x, y=dfp['cuartos_ocupados_pd']/1000,
            name='Cuartos ocupados (miles)',
            marker_color=COL_OCUPADO,
            hovertemplate="Ocupados: %{y:,.0f} mil<extra></extra>"
        ),
        secondary_y=False
    )

    # Línea % (eje derecho)
    fig.add_trace(
        go.Scatter(
            x=x, y=dfp['porc_ocupacion'],
            name='% Ocupación',
            mode='lines+markers',
            line=dict(color=COL_LINEA_OCUPACION, width=3),
            marker=dict(size=7),
            hovertemplate="% Ocupación: %{y:.1f}%<extra></extra>"
        ),
        secondary_y=True
    )

    # Layout limpio
    layout = _base_layout("Actividad hotelera en México (Cuartos promedio diario)")
    layout.update(
        xaxis=dict(title_text="Año", showgrid=False, zeroline=False, showline=False, ticks="outside"),
        yaxis=dict(title_text="Cuartos (miles)", tickformat=",.0f",
                   showgrid=True, gridcolor="#D0D0D0", gridwidth=1, zeroline=False, showline=False),
        yaxis2=dict(title_text="% Ocupación", range=[0, 100],
                    tickformat=",.0f", ticksuffix="%", showgrid=False, zeroline=False, showline=False),
        barmode='group', bargap=0.15, bargroupgap=0.05
    )
    fig.update_layout(**layout)

    st.plotly_chart(fig, use_container_width=True)

    # --- TABLA DE DATOS VISIBLE ---
    st.markdown("**Datos detallados (Histórico)**")
    
    tabla_hist = dfp[['anio', 'cuartos_disponibles_pd', 'cuartos_ocupados_pd', 'porc_ocupacion']].copy()
    tabla_hist.columns = ["Año", "Cuartos Disponibles", "Cuartos Ocupados", "% Ocupación"]
    
    st.dataframe(
        tabla_hist,
        use_container_width=True,
        hide_index=True,
        column_config={
            "Año": st.column_config.NumberColumn(format="%d"),
            "Cuartos Disponibles": st.column_config.NumberColumn(format="%,.0f"),
            "Cuartos Ocupados": st.column_config.NumberColumn(format="%,.0f"),
            "% Ocupación": st.column_config.NumberColumn(format="%.1f%%")
        }
    )

def plot_cat_barras_con_linea(cat_year, cat_df: pd.DataFrame):
    """Gráfica 2: Ocupación por categoría (Barras Cuartos + Línea % Ocupación) CON TABLA FIJA."""
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if cat_df.empty:
        st.warning(f"No hay datos por categoría para el último año con datos ({cat_year}).")
        return

    cats = cat_df['categoria'].astype(str).tolist()
    disp = cat_df['cuartos_disponibles_pd'].values
    ocup = cat_df['cuartos_ocupados_pd'].values
    occp = cat_df['porc_ocupacion'].values

    fig.add_trace(
        go.Bar(
            x=cats, y=disp, name="Cuartos disponibles",
            marker_color=COL_DISPONIBLE,
            hovertemplate="Disponibles: %{y:,.0f}<extra></extra>"
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Bar(
            x=cats, y=ocup, name="Cuartos ocupados",
            marker_color=COL_OCUPADO,
            hovertemplate="Ocupados: %{y:,.0f}<extra></extra>"
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=cats, y=occp, name="% Ocupación",
            mode='lines+markers', line=dict(color=COL_LINEA_OCUPACION, width=3),
            marker=dict(size=7),
            hovertemplate="% Ocupación: %{y:.1f}%<extra></extra>"
        ),
        secondary_y=True
    )

    subtitle = f" — {cat_year}" if cat_year else ""
    layout = _base_layout(f"Ocupación hotelera por categoría{subtitle}")
    layout.update(
        xaxis=dict(title_text="Categoría", showgrid=False, zeroline=False, showline=False, ticks="outside", tickangle=0),
        yaxis=dict(title_text="Cuartos (Promedio diario)", tickformat=",.0f",
                   showgrid=True, gridcolor="#D0D0D0", gridwidth=1, zeroline=False, showline=False),
        yaxis2=dict(title_text="% Ocupación", range=[0, 100],
                    tickformat=",.0f", ticksuffix="%", showgrid=False, zeroline=False, showline=False),
        barmode='group', bargap=0.20, bargroupgap=0.06
    )
    fig.update_layout(**layout)

    st.plotly_chart(fig, use_container_width=True)

    # --- TABLA DE DATOS VISIBLE ---
    st.markdown(f"**Datos detallados (Categoría {cat_year})**")
    
    tabla_cat = cat_df[['categoria', 'cuartos_disponibles_pd', 'cuartos_ocupados_pd', 'porc_ocupacion']].copy()
    tabla_cat.columns = ["Categoría", "Cuartos Disponibles", "Cuartos Ocupados", "% Ocupación"]
    
    st.dataframe(
        tabla_cat,
        use_container_width=True,
        hide_index=True,
        column_config={
            "Categoría": st.column_config.TextColumn(),
            "Cuartos Disponibles": st.column_config.NumberColumn(format="%,.0f"),
            "Cuartos Ocupados": st.column_config.NumberColumn(format="%,.0f"),
            "% Ocupación": st.column_config.NumberColumn(format="%.1f%%")
        }
    )

# -----------------------------------------------------------------------------
# 4. MAIN FLOW STREAMLIT
# -----------------------------------------------------------------------------

st.markdown("### 🏨 Indicadores de Actividad Hotelera (DataTur)")

# Lectura de datos
with st.spinner("1. Descargando y normalizando datos de 70 centros turísticos..."):
    df_70_raw = read_csv_super(URL_70)
    df_70 = std_cols(df_70_raw)

    if URL_HIST:
        df_hist_raw = read_csv_super(URL_HIST)
        df_hist = std_cols(df_hist_raw)
    else:
        df_hist = pd.DataFrame()

if df_70.empty:
    st.error("No se pudo obtener o leer el CSV de actividad hotelera (Base70centros.csv).")
    st.stop()

# 2. Serie anual nacional (Combinar histórico y reciente)
hist_annual = annual_national(df_hist[df_hist['anio']<=2014]) if (not df_hist.empty and 'anio' in df_hist.columns) else pd.DataFrame()
recent_annual = annual_national(df_70[df_70['anio']>=2015])
national_annual = pd.concat([hist_annual, recent_annual], ignore_index=True)

if not national_annual.empty and 'anio' in national_annual.columns:
    national_annual = national_annual.dropna(subset=['anio']).sort_values('anio')
    y0, y1 = int(national_annual['anio'].min()), int(national_annual['anio'].max())
    st.caption(f"Serie histórica nacional disponible: {y0} a {y1}.")
else:
    st.warning("⚠ No se pudo construir la serie anual nacional para graficar el histórico.")

# 3. Datos por categoría
cat_year, cat_df = category_occupancy_latest_year(df_70)

# 4. Tabla Playa vs Ciudades (usando la constante FALLBACK_CENTROS)
tabla_centros_data = []
# Se utiliza datetime.now().year para obtener el año actual y predecir el siguiente
from_year = datetime.now().year
for (tipo, periodo), datos in FALLBACK_CENTROS.items():
    tabla_centros_data.append({
        "Tipo de centro turístico": tipo,
        "Periodo": periodo,
        "Cuartos Disponibles (Promedio diario)": f"{datos['disp']:,}",
        "Ocupación (%)": f"{datos['occ']:.1f}%"
    })
tabla_centros_df = pd.DataFrame(tabla_centros_data)

# 5. Mostrar Gráficas
st.markdown("---")
tab1, tab2 = st.tabs(["📊 Histórico Nacional", "⭐ Ocupación por Categoría"])

with tab1:
    plot_hist_barras_con_linea(national_annual, prefer='quinquenios')
    st.caption("Gráfica: Promedio diario anual de cuartos disponibles y ocupados (eje izquierdo, en miles) vs. Porcentaje de ocupación (eje derecho).")

with tab2:
    plot_cat_barras_con_linea(cat_year, cat_df)
    st.caption(f"Datos agregados por categoría para el último año completo disponible: {cat_year}.")

/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning: 


This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.

  from .kaleido import Kaleido


Leyendo 2015+ desde: https://repodatos.atdt.gob.mx/s_turismo/ocupacion_hotelera/Base70centros.csv

[DEBUG] Columnas 2015+ : ['anio', 'mes', 'tipo_centro', 'centro_turistico', 'categoria', 'cuartos_disponibles_pd', 'cuartos_ocupados_pd', 'llegada_turistas_no_residentes', 'llegada_turistas_residentes', 'turistas_noche_no_residentes', 'turistas_noche_residentes']
[DEBUG] Columnas histórico: []

[DEBUG] Muestra 2015+:
  anio  mes      tipo_centro     centro_turistico   categoria  cuartos_disponibles_pd  cuartos_ocupados_pd llegada_turistas_no_residentes llegada_turistas_residentes turistas_noche_no_residentes turistas_noche_residentes
 2016    1 Centros de Playa Ixtapa - Zihuatanejo 3 estrellas                   10819                 5333                           3888                        2023                        10116                      5175
 2016    1 Centros de Playa Ixtapa - Zihuatanejo 4 estrellas                   45090                 8114                           3941     

⚠ Falló la exportación PNG, reintentando con ajustes de Kaleido...
⚠ No se pudo exportar PNG con Kaleido (ValueError): 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

→ Abre el HTML y usa el botón 'Download PNG' en la barra de herramientas del gráfico.
HTML interactivo guardado: fig_actividad_hotelera_2000_actual.html
[DEBUG] Gráfica por categoría — Último año: 2024


⚠ Falló la exportación PNG, reintentando con ajustes de Kaleido...
⚠ No se pudo exportar PNG con Kaleido (ValueError): 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

→ Abre el HTML y usa el botón 'Download PNG' en la barra de herramientas del gráfico.
HTML interactivo guardado: fig_ocupacion_por_categoria_actual.html

Listo ✅
- fig_actividad_hotelera_2000_actual.png (y fig_actividad_hotelera_2000_actual.html)
- fig_ocupacion_por_categoria_actual.png  (y fig_ocupacion_por_categoria_actual.html)
